In [1]:
from transformers import ViTImageProcessor, ViTForImageClassification
from tqdm import tqdm
import numpy as np
from datasets import load_dataset
import matplotlib.pyplot as plt
import torch
import os
import pickle

external_path=''

In [2]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('aaraki/vit-base-patch16-224-in21k-finetuned-cifar10')

In [3]:
dataset = load_dataset("uoft-cs/cifar10")

train_set = dataset["train"]
test_set = dataset["test"]

: 

In [4]:
categories=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [5]:
fig,axs=plt.subplots(nrows=2,ncols=5,layout='tight')
fig.set_figheight(2)
fig.set_figwidth(5)
count=0
plotted_samples=[]
for instance in test_set:
    if not(instance['label'] in plotted_samples):
        image=np.array(instance['img'])

        axs[count//5,count%5].imshow(image,aspect='auto')
        axs[count//5,count%5].axis('off')
        axs[count//5,count%5].set_title(categories[instance['label']],fontsize=8)
        count+=1
        plotted_samples.append(instance['label'])

In [7]:
progress_bar=tqdm(categories)
class_test_indices={category:[] for category in categories}
for category in progress_bar:
    if not(os.path.exists(f'{external_path}\\{category}')):
        os.mkdir(f'{external_path}\\class_token_activations\\{category}')
    
    total_num_instances=0
    acc_count=0

    save_block=1
    category_class_token_activations=[]
    first_instance=True
    num_instances=0
    
    for n,test_instance in enumerate(test_set):

        if categories[test_instance['label']]!=category:
            continue

        image=np.array(test_instance['img'])

        model_input = processor(images=image, return_tensors="pt")
        model_output = model(**model_input,output_hidden_states=True)
        
        predicted_category=categories[model_output.logits.argmax(-1).item()]

        if predicted_category==category:

            class_test_indices[category]+=[n]

            acc_count+=1
            for k, hidden_state in enumerate(model_output.hidden_states):
                if first_instance:
                    category_class_token_activations.append(hidden_state[:,0,:].unsqueeze(0))
                else:
                    category_class_token_activations[k]=torch.cat([category_class_token_activations[k],hidden_state[:,0,:].unsqueeze(0)])
            first_instance=False
        num_instances+=1
        total_num_instances+=1
        progress_bar.set_description(f'Category {category}...{total_num_instances}/1000...{acc_count/total_num_instances:.4f}')

        if num_instances==100:
            for layer,activations in enumerate(category_class_token_activations):
                torch.save(activations,f'{external_path}\\class_token_activations\\{category}\\layer{layer}_{save_block}.pt')

            category_class_token_activations=[]
            first_instance=True
            num_instances=0
            save_block+=1

    training_log=open(f'{external_path}\\class_token_activations\\{category}\\training_log.txt','w')
    training_log.write(str(acc_count))
    training_log.close()

class_test_indices_file=open(f'{external_path}\\class_correctly_classified_indices','wb')
pickle.dump(class_test_indices,class_test_indices_file)
class_test_indices_file.close()

Category truck...1000/1000...0.9720: 100%|██████████| 10/10 [2:16:43<00:00, 820.33s/it]
